In [2]:
'''
Author: Tytrez Dixon
Date Written: 3/9/2025
Date Updated: 3/21/2025

This program acts as a series of simple tests of the backtesting.py library 
and its functions.
'''

'\nAuthor: Tytrez Dixon\nDate Written: 3/9/2025\nDate Updated: 3/11/2025\n\nThis program acts as a series of simple tests of the backtesting.py library \nand its functions.\n'

In [3]:
# Make necessary imports.
from backtesting import Backtest, Strategy
from backtesting.test import GOOG
import yfinance as yf
import pandas as pd
import ta
from backtesting.lib import crossover
import yfinance as yf

/Users/tytrezdixon/Desktop/Trading Bot Project/.venv/lib/python3.12/site-packages/backtesting/_plotting.py:54: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [7]:
df = yf.download('GOOG', start='2018-01-01')

print(df.head())  # Print first few rows
print(df.columns)  # Print column names
print(df.dtypes)   # Print data types

print(list(df.columns))

[*********************100%***********************]  1 of 1 completed

Price           Close       High        Low       Open    Volume
Ticker           GOOG       GOOG       GOOG       GOOG      GOOG
Date                                                            
2018-01-02  52.998970  53.095513  52.015131  52.169897  24752000
2018-01-03  53.868855  54.058455  52.909896  52.964637  28604000
2018-01-04  54.063927  54.420738  53.944592  54.143551  20092000
2018-01-05  54.851692  54.952218  54.342602  54.442133  25582000
2018-01-08  55.086082  55.301561  54.821338  54.851692  20952000
MultiIndex([( 'Close', 'GOOG'),
            (  'High', 'GOOG'),
            (   'Low', 'GOOG'),
            (  'Open', 'GOOG'),
            ('Volume', 'GOOG')],
           names=['Price', 'Ticker'])
Price   Ticker
Close   GOOG      float64
High    GOOG      float64
Low     GOOG      float64
Open    GOOG      float64
Volume  GOOG        int64
dtype: object
[('Close', 'GOOG'), ('High', 'GOOG'), ('Low', 'GOOG'), ('Open', 'GOOG'), ('Volume', 'GOOG')]


In [10]:
# Print info regarding the Google stock.
print(GOOG)

              Open    High     Low   Close    Volume
2004-08-19  100.00  104.06   95.96  100.34  22351900
2004-08-20  101.01  109.08  100.50  108.31  11428600
2004-08-23  110.75  113.48  109.05  109.40   9137200
2004-08-24  111.24  111.60  103.57  104.87   7631300
2004-08-25  104.96  108.00  103.88  106.00   4598900
...            ...     ...     ...     ...       ...
2013-02-25  802.30  808.41  790.49  790.77   2303900
2013-02-26  795.00  795.95  784.40  790.13   2202500
2013-02-27  794.80  804.75  791.11  799.78   2026100
2013-02-28  801.10  806.99  801.03  801.20   2265800
2013-03-01  797.80  807.14  796.15  806.19   2175400

[2148 rows x 5 columns]


In [11]:
# Create a class for the SMA strategy.
class SMAcross (Strategy):

    n1 = 50
    n2 = 100

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(ta.trend.sma_indicator, pd.Series(close), self.n1)
        self.sma2 = self.I(ta.trend.sma_indicator, pd.Series(close), self.n2)

    # Buy/Sell when a crossover occurs.
    def next (self):
        
        if crossover(self.sma1, self.sma2):
            self.buy()

        elif crossover(self.sma2, self.sma1):
            self.sell()
 

In [23]:
# Download stock data from Yahoo Finance
df = yf.download('GOOG', start='2018-01-01')

df = df.apply(pd.to_numeric, errors='coerce')

# Check if DataFrame has a MultiIndex and flatten it
if isinstance(df.columns, pd.MultiIndex):
    df.columns = df.columns.get_level_values(0)  # Take first level

# Rename columns to match Backtest expected format
df.rename(columns={'Adj Close': 'Close'}, inplace=True)

# Ensure the index is a DateTimeIndex
df.index = pd.to_datetime(df.index)

# Display first few rows of the DataFrame
df.head()



[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Date,,,,,
2018-01-02,52.998974,53.095517,52.015135,52.169901,24752000
2018-01-03,53.868851,54.058451,52.909892,52.964633,28604000
2018-01-04,54.063927,54.420738,53.944592,54.143551,20092000
2018-01-05,54.851700,54.952225,54.342610,54.442141,25582000
2018-01-08,55.086086,55.301565,54.821341,54.851696,20952000


In [24]:
print(df.head())   # Show first few rows
print(df.columns)  # Show column names
print(df.dtypes)   # Show column data types

Price           Close       High        Low       Open    Volume
Date                                                            
2018-01-02  52.998974  53.095517  52.015135  52.169901  24752000
2018-01-03  53.868851  54.058451  52.909892  52.964633  28604000
2018-01-04  54.063927  54.420738  53.944592  54.143551  20092000
2018-01-05  54.851700  54.952225  54.342610  54.442141  25582000
2018-01-08  55.086086  55.301565  54.821341  54.851696  20952000
Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Price')
Price
Close     float64
High      float64
Low       float64
Open      float64
Volume      int64
dtype: object


In [25]:
# Run Backtest
bt = Backtest(df, SMAcross, cash=100000, commission=0.002, exclusive_orders=True)
output = bt.run()

output

Start                     2018-01-02 00:00:00
End                       2025-03-21 00:00:00
Duration                   2635 days 00:00:00
Exposure Time [%]                     89.2011
Equity Final [$]                 185513.94086
Equity Peak [$]                   274424.8056
Commissions [$]                     7273.2544
Return [%]                           85.51394
Buy & Hold Return [%]               209.54642
Return (Ann.) [%]                     8.95877
Volatility (Ann.) [%]                32.78057
CAGR [%]                              6.08803
Sharpe Ratio                           0.2733
Sortino Ratio                         0.42563
Calmar Ratio                          0.18022
Max. Drawdown [%]                   -49.70991
Avg. Drawdown [%]                    -5.35472
Max. Drawdown Duration     1062 days 00:00:00
Avg. Drawdown Duration       45 days 00:00:00
# Trades                                   14
Win Rate [%]                         35.71429
Best Trade [%]                    